In [1]:
from sympy import Symbol, S,symbols,Mul,Function,conjugate,Add,diag,I,integrate,cos,sin,diag,pi,nsimplify,factor
from sympy.tensor.tensor import TensorHead,Tensor,TensorIndexType, tensor_indices, tensor_heads,TensorIndex,TensorSymmetry
from sympy.core.expr import Expr

Lorentz = TensorIndexType('Lorentz', dummy_name='L', dim=4)
Lg = Lorentz.metric

In [2]:
def g(id1,id2):
    m0, m1 = tensor_indices('m0,m1', Lorentz)
    p,q=1,1;
    
    if not isinstance(id1,TensorIndex):
        p = tensor_heads(id1.name, [Lorentz])(-m0)
        id1=m0
        
    if not isinstance(id2,TensorIndex):
        q = tensor_heads(id2.name, [Lorentz])(-m1)
        id2=m1
        
    return (Lg(id1,id2)*p*q)
    
class Spinor(Symbol):
    def __new__(cls,momentum,mass,polarization,**kwargs):
        obj = super().__new__(cls,momentum.name,**kwargs,commutative=False)
        obj.momentum = momentum
        obj.mass = mass
        obj.polarization=polarization
        return obj
    def __init__(self,momentum,mass,polarization):
        self.name =f"u({momentum})" 



class GammaSlash(Symbol):  
    def __new__(cls,name,*args,**kwargs):
        obj_name=name,
        obj = super().__new__(cls,name.name,*args,**kwargs,commutative=False)
        obj.index = name
        return obj
    def __init__(self,name):
        self.name =f" gs({self.index.name})"
        self.is_gamma5=False
        self.is_gs=True
         
class GammaMatrix(Symbol):
    def __new__(cls,name,*args,**kwargs):
        obj_name=f"gamma^{name.name}" if name.is_up else f"gamma_{name.name}"
        obj = super().__new__(cls,obj_name,*args,**kwargs,commutative=False)
        obj.index = name
        return obj
        
    def __init__(self,name):
        self.is_gamma5=False
        self.is_gs=False
            
class GammaMatrix5(Symbol):
    def __new__(cls,*args,**kwargs):
        obj = super().__new__(cls,name='gamma^5',*args,**kwargs,commutative=False)
        return obj
    
    def __init__(self):
        self.name =f"gamma^5"
        self.is_gamma5=True
        self.is_gs=False
        
    def __pow__(self, power):
        if power == 2:
            return S.One
        elif power == 3:
            return self  # G5^3 = -G5
        else:
            raise ValueError(f"Power {power} not implemented for {self.name}")
    def __repr__(self):
        return self.name

    def __mul__(self, other):
        
        if other.__class__.__name__=='GammaMatrix5':
            return S.One
        return super().__mul__(other)
G5=GammaMatrix5()
PL=(S.One-G5)/2
PR=(S.One+G5)/2

class GC:         
    def __init__(self,left_spinor,matrix,right_spinor):
        self.left_spinor = left_spinor
        self.matrix = matrix
        self.right_spinor = right_spinor
        
    def __repr__(self):
        return f"GC({self.left_spinor}{self.matrix}{self.right_spinor})"
    
    def __mul__(self, other):
        L_contract=self.right_spinor.momentum==other.left_spinor.momentum;
        R_contract=self.left_spinor.momentum==other.right_spinor.momentum;
        if L_contract and (not R_contract):
            p1=(GammaSlash(self.right_spinor.momentum)+self.right_spinor.mass)*(1+self.right_spinor.polarization*G5)
            return GC(self.left_spinor,self.matrix*p1*other.matrix,other.right_spinor)
        if (not L_contract) and  R_contract:
            p1=(GammaSlash(self.left_spinor.momentum)+self.left_spinor.mass)
            return GC(other.left_spinor,other.matrix*p1*self.matrix,self.right_spinor)*(1+self.right_spinor.polarization*G5)
        if L_contract and R_contract:
            p1=(GammaSlash(other.right_spinor.momentum)+S.One*other.right_spinor.mass)*(1+other.right_spinor.polarization*G5)
            p2=(GammaSlash(self.right_spinor.momentum)+S.One*self.right_spinor.mass)*(1+self.right_spinor.polarization*G5)
            return trace(other.matrix*p1*self.matrix*p2)
            
        return Mul(self, other, evaluate=False)

def reorder_with_eps(expr):    
    terms=expr.args
    reordered_terms = []
    if len(terms)==0 and not isinstance(expr,Symbol):
        return expr

    for index, term in enumerate(terms):
        if term.__class__.__name__=='LeviCivitaT':
            reordered_terms.insert(0, term)
        else:
            reordered_terms.append(term)
    # Combine terms back into an expression
    reordered_expr=1
    for x in reordered_terms:
        reordered_expr*=x
    return reordered_expr
    
def reorder_with_sign(expr):   
    
    cof=Mul(*list(filter(lambda x:not hasattr(x,'is_gs'),  expr.args)))
    terms=list(filter(lambda x:hasattr(x,'is_gs'),  expr.args))
    if len(terms)==0 and not isinstance(expr,Symbol):
        return expr
    reordered_terms = []
    # Iterate through terms to move `target` to the front
    n=0;
    for index, term in enumerate(terms):
        if term.__class__.__name__=='GammaMatrix5':
            # If target is found, prepend it and adjust the sign
            reordered_terms.insert(0, term)
            
            cof *= (-1)**(index-n)  # Flip the sign for each swap
            n+=1;
        else:
            reordered_terms.append(term)
    
    # Combine terms back into an expression
    reordered_expr = Mul(*reordered_terms)
    return cof * reordered_expr
    
def GMSimplify(GMs):
    expandsion=(GMs*S.One).expand()
    if not isinstance(expandsion,Mul):
        Z=Add(*[reorder_with_sign(gms_arg) for gms_arg in list(expandsion.args)])
        return Z
    else :
        return reorder_with_sign(expandsion)

def Trace_Sigle(expandsion):
    n= sum([1 for x in list(expandsion.args) if hasattr(x,'index') ])
    m= sum([1 for x in list(expandsion.args) if hasattr(x,'is_gamma5') and x.is_gamma5])
    coeff=(Mul(*filter(lambda x: not hasattr(x,'index')  and not (hasattr(x,'is_gamma5') and x.is_gamma5), expandsion.args) ))
    if(m==0):
        if n==2:
            indies=list(map(lambda x:x.index,filter(lambda x:hasattr(x,'index'),list(expandsion.args))))
            
            return 4*coeff*g(indies[0],indies[1])
        if n==4:
            indies=list(map(lambda x:x.index,filter(lambda x:hasattr(x,'index'),list(expandsion.args))))
            return 4*coeff*(g(indies[0],indies[1])*g(indies[2],indies[3])+g(indies[0],indies[3])*g(indies[1],indies[2])-g(indies[0],indies[2])*g(indies[1],indies[3]))
    else:
        if n==4:
            indies=list(map(lambda x:x.index,filter(lambda x:hasattr(x,'index'),list(expandsion.args))))
            #print(LeviCivitaT(1,self_indices=(indies[0],indies[1],indies[2],indies[3])))
            m0, m1,m2, m3= tensor_indices('m0,m1,m2,m3', Lorentz)
            p0,p1,p2,p3=1,1,1,1;
            id0,id1,id2,id3=indies
            
            if not isinstance(id0,TensorIndex):
                p0 = tensor_heads(id0.name, [Lorentz])(-m0)
                id1=m0
            if not isinstance(id1,TensorIndex):
                p1 = tensor_heads(id1.name, [Lorentz])(-m1)
                id1=m1
            if not isinstance(id2,TensorIndex):
                p2 = tensor_heads(id2.name, [Lorentz])(-m2)
                id2=m2
            if not isinstance(id3,TensorIndex):
                p3 = tensor_heads(id3.name, [Lorentz])(-m3)
                id3=m3
            
            return 4j*coeff*LeviCivitaT(1,self_indices=(id0,id1,id2,id3))*p0*p1*p2*p3
        
        if n==6:
            indies=list(map(lambda x:x.index,filter(lambda x:hasattr(x,'index'),list(expandsion.args))))
            m0,m1,m2,m3,m4,m5= tensor_indices('m0,m1,m2,m3,m4,m5', Lorentz)
            p0,p1,p2,p3,p4,p5=1,1,1,1,1,1
            id0,id1,id2,id3,id4,id5=indies
            
            if not isinstance(id0,TensorIndex):
                p0 = tensor_heads(id0.name, [Lorentz])(-m0)
                id1=m0
            if not isinstance(id1,TensorIndex):
                p1 = tensor_heads(id1.name, [Lorentz])(-m1)
                id1=m1
            if not isinstance(id2,TensorIndex):
                p2 = tensor_heads(id2.name, [Lorentz])(-m2)
                id2=m2
            if not isinstance(id3,TensorIndex):
                p3 = tensor_heads(id3.name, [Lorentz])(-m3)
                id3=m3
            if not isinstance(id4,TensorIndex):
                p4 = tensor_heads(id4.name, [Lorentz])(-m4)
                id4=m4
            if not isinstance(id5,TensorIndex):
                p5 = tensor_heads(id5.name, [Lorentz])(-m5)
                id5=m5
            S=g(id0,id1)*LeviCivitaT(1,self_indices=(id2,id3,id4,id5))
            S+=-g(id0,id2)*LeviCivitaT(1,self_indices=(id1,id3,id4,id5))
            S+=g(id1,id2)*LeviCivitaT(1,self_indices=(id0,id3,id4,id5))
            
            S+=g(id3,id4)*LeviCivitaT(1,self_indices=(id0,id1,id2,id5))
            S+=-g(id3,id5)*LeviCivitaT(1,self_indices=(id0,id1,id2,id4))
            S+=g(id4,id5)*LeviCivitaT(1,self_indices=(id0,id1,id2,id3))
            return -4j*coeff*S*p0*p1*p2*p3*p4*p5
    return 0

def trace(GMs):
    expandsion=GMSimplify(GMs)

    if not isinstance(expandsion,Mul):
       return Add(*list(map(lambda x:Trace_Sigle(x),list(expandsion.args))))
    else :
        return Trace_Sigle(expandsion)
def conj(GCm):
    m=(GCm.matrix) if GCm.matrix.is_symbol else Mul(* reversed(list(GCm.matrix.args)))
    return GC(GCm.right_spinor,m.subs(G5,-G5),GCm.left_spinor)
def delta(id1,id2):
    m0 = tensor_indices('m0', Lorentz)
    
    return (Lg(id1,m0)*Lg(id2,-m0)).contract_metric(Lg)
class LeviCivitaT(Tensor):
    def __init__(self,name,self_indices,**is_canon_bp):
        self.name=1;
    def __new__(cls,name,self_indices,**is_canon_bp):
        instance = super().__new__(cls,tensor_head=TensorHead('epsilon', [Lorentz]*4,TensorSymmetry.fully_symmetric(-4)),indices=self_indices)
        return instance
    
    def __mul__(self, other):
        
        if other.__class__.__name__=='LeviCivitaT':
            
            self_indices=list(map(lambda x:-x,self.get_indices()))
            other_indices=other.get_indices()
            combined_indices =  self_indices+ other_indices
            self_free_indices = [idx for idx in self_indices if combined_indices.count(idx) == 1]
            other_free_indices = [idx for idx in other_indices if combined_indices.count(idx) == 1]
            self_dummy_indices = [idx for idx in self_indices if combined_indices.count(idx) == 2]
            other_dummy_indices = [idx for idx in other_indices if combined_indices.count(idx) == 2]

            if(len(self_free_indices)==2 and len(other_free_indices)==2):
                
                osfids=list(map(lambda x:-x,self_free_indices))
                f1=(-1)**(self_indices.index(self_dummy_indices[0])-0)*(-1)**(self_indices.index(self_dummy_indices[1])-1)
                f2=(-1)**(other_indices.index(other_dummy_indices[0])-0)*(-1)**(other_indices.index(other_dummy_indices[1])-1)
                dd1=delta(osfids[0],other_free_indices[0])*delta(osfids[1],other_free_indices[1])
                dd2=delta(osfids[0],other_free_indices[1])*delta(osfids[1],other_free_indices[0])
                return f1*f2*(-2)*(dd1-dd2)
            
            if(len(self_free_indices)==1 and len(other_free_indices)==1):
                osfids=list(map(lambda x:-x,self_free_indices))
                f1=(-1)**(self_indices.index(self_dummy_indices[0])-0)*(-1)**(self_indices.index(self_dummy_indices[1])-1)
                f2=(-1)**(other_indices.index(other_dummy_indices[0])-0)*(-1)**(other_indices.index(other_dummy_indices[1])-1)
                return f1*f2*(-6)*delta(osfids[0],other_free_indices[0])
        return super().__mul__(other)#.canon_bp()
       
def Evl(epx):
    return Add(*[reorder_with_eps(x) for x in epx.expand().args]).canon_bp().contract_metric(Lg)    




In [5]:
def replaceTensor(term):
    tensors=list(filter(lambda x:isinstance(x,Tensor),term.args))
    my_dict={}
    my_dict[Lorentz] = diag(1, -1, -1, -1)
    for x in tensors:
        my_dict.setdefault(x,r[x.head.name])
    return term.replace_with_arrays(my_dict)

def T(name):
  return tensor_heads(name, [Lorentz])
    
def cous_subs(exper,tensor_name_to_replace,rul):
    for x in exper.args:
        if isinstance(x,Tensor) and x.head.name==tensor_name_to_replace:
            exper=exper.subs(x,rul( x.indices[0]))
    return exper
def innerp_subs(exper,in1_a,in1_b):
    i0 = tensor_indices('i0', Lorentz)
    sing=1
    term=momentum_conservations(i0).subs(T(in1_a)(i0),0).subs(T(in1_b)(i0),0)*momentum_conservations(-i0).subs(T(in1_a)(-i0),0).subs(T(in1_b)(-i0),0)
    term+=-T(in1_a)(i0)*T(in1_a)(-i0)/2
    term+=-T(in1_b)(i0)*T(in1_b)(-i0)/2
    
    sing*=momentum_conservations_signs[in1_a]
    sing*=momentum_conservations_signs[in1_b]
    
    for x in exper.args:
        
        if isinstance(x,Tensor) and x.head.name==in1_a:
            exper=exper.subs(x,1)  
            
        if isinstance(x,Tensor) and x.head.name==in1_b:
            exper=exper.subs(x,1)
            
    return exper*sing*term

In [6]:
mu,nu,alpha,beta,rho,sigma = tensor_indices('mu,nu,alpha,beta,rho,sigma', Lorentz)

n1,n2,n3,n=symbols("n1,n2,n3,n")

E_nub,p_nub,theta_nub,phi_nub=symbols(r"E_{\bar\nu} p_{\bar\nu} \theta_{\bar\nu} \phi_{\bar\nu}")
E_nu,p_nu,theta_nu,phi_nu=symbols(r"E_\nu p_\nu \theta_\nu \phi_\nu")
E_e,p_e,m_e,theta_e,phi_e=symbols(r"E_e p_e m_e \theta_e \phi_e")
E_mu,p_mu,m_mu=symbols(r"E_\mu p_\mu m_mu")

r={
     r'p_e':[E_e,p_e*cos(phi_e)*sin(theta_e),p_e*sin(phi_e)*sin(theta_e),p_e*cos(theta_e)]
    ,r'n':[0,n1,n2,n3]
    ,r'p_{\bar\nu}':[E_nub,p_nub*cos(phi_nub)*sin(theta_nub),p_nub*sin(phi_nub)*sin(theta_nub),p_nub*cos(theta_nub)]
    ,r'p_\mu':[m_mu,0,0,0]
    ,r'p_\nu':[E_nu,p_nu*cos(phi_nu)*sin(theta_nu),p_nu*sin(phi_nu)*sin(theta_nu),p_nu*cos(theta_nu)]
}


u1=Spinor(p_e,m_e,0)
u2=Spinor(p_nu,0,0)
u3=Spinor(p_nub,0,0)
u4=Spinor(p_mu,m_mu,GammaSlash(n))

In [7]:
momentum_conservations_signs={r'p_\mu':+1,r'p_\nu':-1,r'p_{\bar\nu}':-1,r'p_e':-1}

In [8]:
momentum_conservations=lambda i: Add(*[ sign*T(key)(i) for key, sign in momentum_conservations_signs.items()])
momentum_conservations(mu)

-p_\nu(mu) - p_e(mu) - p_{\bar\nu}(mu) + p_\mu(mu)

In [13]:
T1=(1j/2)*(GammaMatrix(alpha)*GammaMatrix(beta)-GammaMatrix(beta)*GammaMatrix(alpha))
conj_T1=(1j/2)*(GammaMatrix(-alpha)*GammaMatrix(-beta)-GammaMatrix(-beta)*GammaMatrix(-alpha))
T2=(1j/2)*(GammaMatrix(rho)*GammaMatrix(sigma)-GammaMatrix(sigma)*GammaMatrix(rho))
conj_T2=(1j/2)*(GammaMatrix(-rho)*GammaMatrix(-sigma)-GammaMatrix(-sigma)*GammaMatrix(-rho))

g1=GammaMatrix(mu)
g2=GammaMatrix(nu)
conj_g1=GammaMatrix(-mu)
conj_g2=GammaMatrix(-nu)


In [16]:
M1=conj(GC(u1,PL*g1,u2))*GC(u1,PL*g2,u2)
M2=conj(GC(u3,conj_g1*PL,u4))*GC(u3,conj_g2*PL,u4)
Evl(M1*M2)

16.0*p_\mu(L_0)*p_\nu(L_1)*p_e(-L_0)*p_{\bar\nu}(-L_1) + 16.0*m_mu*n(L_0)*p_\nu(L_1)*p_e(-L_0)*p_{\bar\nu}(-L_1)

In [18]:
"""
Ms=[GC(u1,PR*S.One,u2),GC(u3,S.One*PR,u4)]
Mv=[GC(u1,PL*g1,u2),GC(u3,conj_g1*PL,u4)]

Z1=conj(Ms[0])*Mv[0]
Z2=conj(Ms[1])*Mv[1]
Z3=Z1*Z2
Z3.contract_metric(Lg) 
"""

'\nMs=[GC(u1,PR*S.One,u2),GC(u3,S.One*PR,u4)]\nMv=[GC(u1,PL*g1,u2),GC(u3,conj_g1*PL,u4)]\n\nZ1=conj(Ms[0])*Mv[0]\nZ2=conj(Ms[1])*Mv[1]\nZ3=Z1*Z2\nZ3.contract_metric(Lg) \n'

In [20]:
"""
mu,Ee,x=symbols(r'm_\mu E_e x')
cosnue=(mu**2-2*mu*(Ee+x)+2*Ee*x)/(2*Ee*x)
s=x*(Ee+(mu-Ee)*cosnue)
s.expand()
integrate(s,(x,mu/2-Ee,mu/2)).expand().simplify()
"""

"\nmu,Ee,x=symbols(r'm_\\mu E_e x')\ncosnue=(mu**2-2*mu*(Ee+x)+2*Ee*x)/(2*Ee*x)\ns=x*(Ee+(mu-Ee)*cosnue)\ns.expand()\nintegrate(s,(x,mu/2-Ee,mu/2)).expand().simplify()\n"

In [56]:
Ms=[GC(u1,PR*S.One,u2),GC(u3,S.One*PR,u4)]
Mv=[GC(u1,PL*g1,u2),GC(u3,conj_g1*PL,u4)]

Z1=conj(Ms[0])*Mv[0]
Z2=conj(Ms[1])*Mv[1]
Z3=Z1*Z2

In [58]:
Z3

2*m_e*metric(L_0, L_1)*p_\nu(-L_1)*((-2)*(metric(-L_0, L_2)*p_\mu(-L_2)*metric(L_3, L_4)*n(-L_3)*p_{\bar\nu}(-L_4) + metric(-L_0, L_2)*p_{\bar\nu}(-L_2)*metric(L_3, L_4)*p_\mu(-L_3)*n(-L_4) + (-1)*metric(-L_0, L_2)*n(-L_2)*metric(L_3, L_4)*p_\mu(-L_3)*p_{\bar\nu}(-L_4)) + 2*m_mu*metric(-L_0, L_2)*p_{\bar\nu}(-L_2) + (-2.0)*I*epsilon(-L_0, L_2, L_3, L_4)*p_\mu(-L_2)*n(-L_3)*p_{\bar\nu}(-L_4))

In [61]:
Y1=Evl(Z3)

In [67]:
Y1.args[3]

4*m_e*n(L_0)*p_\mu(L_1)*p_\nu(-L_0)*p_{\bar\nu}(-L_1)

In [69]:
# in order to reduce the number of variable in angles as less as possible, We replace expansion in terms of p_mu at rest frame which has vec{p}_mu=0
Y2=innerp_subs(Y1.args[3],r'p_\nu',r'p_{\bar\nu}')

In [99]:
Y2=Y1.args[3]
isinstance(Y2.expand(),Add)

False

In [101]:
#to substitute the fourvector into matrix form. 
if  isinstance(Y2.expand(),Mul):
    Y3=Add(* [replaceTensor(x) for x in Y2.expand().args])
else:
    Y3=replaceTensor(Y2.expand())

In [103]:
Y3

4*m_e*(E_{\bar\nu}*m_mu*n1*p_\nu*sin(\theta_\nu)*cos(\phi_\nu) + E_{\bar\nu}*m_mu*n2*p_\nu*sin(\phi_\nu)*sin(\theta_\nu) + E_{\bar\nu}*m_mu*n3*p_\nu*cos(\theta_\nu))

In [105]:
#intergrate the phase space and substitute the momentum in term of known energy and mass for later intergration 
Y4=integrate(integrate(Y3,(phi_nu,0,2*pi)),(phi_e,0,2*pi)).subs(p_e,E_e).subs(p_nub,E_nub).subs(p_nu,E_nu).subs(E_nub,m_mu-E_nu-E_e)

In [107]:
Y4

2*pi*(4*E_\nu*m_e*m_mu*n2*(-E_\nu - E_e + m_mu)*sin(\theta_\nu) + 4*m_e*(-E_\nu*m_mu*n2*(-E_\nu - E_e + m_mu)*sin(\theta_\nu) + 2*pi*E_\nu*m_mu*n3*(-E_\nu - E_e + m_mu)*cos(\theta_\nu)))

In [109]:
"""
in case there is the factor of cos(theta_nu), we can use 
the momentum conservation to instead of the terms of energy and mass
"""
Y5=Y4.subs(cos(theta_nu),(m_mu**2-2*m_mu*(E_e+E_nu)+2*E_e*E_nu)/(2*E_e*E_nu))

In [111]:
Y5

2*pi*(4*E_\nu*m_e*m_mu*n2*(-E_\nu - E_e + m_mu)*sin(\theta_\nu) + 4*m_e*(-E_\nu*m_mu*n2*(-E_\nu - E_e + m_mu)*sin(\theta_\nu) + pi*m_mu*n3*(-E_\nu - E_e + m_mu)*(2*E_\nu*E_e + m_mu**2 - 2*m_mu*(E_\nu + E_e))/E_e))

In [113]:
Y6=integrate(Y5,(E_nu,m_mu/2-E_e,m_mu/2))

In [115]:
factor(nsimplify(Y6.simplify()))

8*pi**2*E_e**2*m_e*m_mu*n3*(E_e - m_mu)/3